# Coding Challenge Skynopy Mission Manager

This notebook analyzes satellite access for:
1. **Ground Stations**: Communication stations with standard visibility criteria
2. **Zone of Interest (France)**: Zone of interest for the client. NO contact shall be done when the satellite is in this zone.

This notebook is inspired from OSTK library : https://open-space-collective.github.io/open-space-toolkit-astrodynamics/_build/html/_notebooks/Access%20Computation.html

In [ ]:
import pandas as pd
import plotly.graph_objs as go

from ostk.mathematics.object import RealInterval
from ostk.physics import Environment
from ostk.physics.unit import Length
from ostk.physics.unit import Angle
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import Duration
from ostk.physics.time import Interval
from ostk.physics.time import DateTime
from ostk.physics.time import Time
from ostk.physics.coordinate.spherical import LLA
from ostk.astrodynamics.trajectory import Orbit
from ostk.astrodynamics.trajectory.orbit.model import Kepler
from ostk.astrodynamics.trajectory.orbit.model.kepler import COE
from ostk.astrodynamics.trajectory.orbit.model import SGP4
from ostk.astrodynamics.trajectory.orbit.model.sgp4 import TLE
from ostk.astrodynamics.access import Generator as AccessGenerator
from ostk.astrodynamics.access import AccessTarget
from ostk.astrodynamics.access import VisibilityCriterion
from ostk.astrodynamics.utilities import compute_trajectory_geometry
from ostk.astrodynamics.utilities import compute_time_lla_aer_coordinates

## 1. Setup and Imports

In [ ]:

environment = Environment.default(True)  # Set global environment instance as the default instance 
earth = environment.access_celestial_object_with_name("Earth")

## 2. Environment and Visibility Criteria

In [ ]:
# Standard visibility criterion for ground stations (elevation >= 5°)
visibility_criterion = VisibilityCriterion.from_aer_interval(
    azimuth_interval=RealInterval.closed(0.0, 360.0),  # [deg]
    elevation_interval=RealInterval.closed(5.0, 90.0),  # [deg]
    range_interval=RealInterval.closed(0.0, 10000e3),  # [m]
)

# Special visibility criterion for zone of interest (France) 
visibility_criterion_france = VisibilityCriterion.from_aer_interval(
    azimuth_interval=RealInterval.closed(0.0, 360.0),  # [deg]
    elevation_interval=RealInterval.closed(30.0, 90.0),  # [deg]
    range_interval=RealInterval.closed(0.0, 10000e3),  # [m]
)

## 3. Ground Stations Configuration

In [ ]:
import yaml

# Load stations from YAML file
with open('stations.yaml', 'r') as f:
    stations_data = yaml.safe_load(f)

# Create access targets for ground stations (excluding France, which is the zone of interest)
access_targets = []
ground_station_names = []

for station_name, station_info in stations_data['stations'].items():
    if station_name == "France":
        continue  # Skip France - it will be handled separately as zone of interest
    
    lla_coords = station_info['lla']
    access_target = AccessTarget.from_lla(
        visibility_criterion=visibility_criterion,
        lla=LLA(
            Angle.degrees(lla_coords[0]),  # latitude
            Angle.degrees(lla_coords[1]),  # longitude
            Length.meters(lla_coords[2])   # altitude
        ),
        celestial=earth,
    )
    access_targets.append(access_target)
    ground_station_names.append(station_name)

print(f"Ground Stations configured: {ground_station_names}")

## 4. Zone of Interest (France) Configuration

In [ ]:
# Configure France as a special zone of interest 
lla_france = stations_data['stations']["France"]["lla"]
zone_of_interest = AccessTarget.from_lla(
    visibility_criterion=visibility_criterion_france,
    lla=LLA(
        Angle.degrees(lla_france[0]),  # latitude
        Angle.degrees(lla_france[1]),  # longitude
        Length.meters(lla_france[2])   # altitude
    ),
    celestial=earth,
)


## 5. Satellite Orbit Definition

In [ ]:
epoch = Instant.date_time(DateTime(2018, 1, 1, 0, 0, 0), Scale.UTC)

satellite_orbit = Orbit.sun_synchronous(
    epoch, Length.kilometers(500.0), Time(12, 0, 0), earth
)

In [ ]:
start_instant = Instant.date_time(DateTime.parse("2018-01-01 00:00:00"), Scale.UTC)
end_instant = Instant.date_time(DateTime.parse("2018-01-10 00:00:00"), Scale.UTC)

interval = Interval.closed(start_instant, end_instant)

## 6. Analysis Time Interval

In [ ]:
access_generator = AccessGenerator(environment=environment)


## 7. Ground Stations Access Computation

In [ ]:
# Compute accesses for ground stations
accesses = access_generator.compute_accesses(
    interval=interval,
    access_targets=access_targets,
    to_trajectory=satellite_orbit,
)

assert len(accesses) == len(access_targets)  # a list of accesses per target

# Build accesses data for ground stations
accesses_data = []
for i, target_accesses in enumerate(accesses):
    station_name = ground_station_names[i]
    for access in target_accesses:
        accesses_data.append((
            str(access.get_type()),
            repr(access.get_acquisition_of_signal()),
            repr(access.get_time_of_closest_approach()),
            repr(access.get_loss_of_signal()),
            float(access.get_duration().in_seconds()),
            station_name,
        ))

print(f"Total ground station accesses computed: {len(accesses_data)}")

### Ground Stations Access Results

In [ ]:
# Create DataFrame for ground stations accesses
accesses_df = pd.DataFrame(
    data=accesses_data,
    columns=["Type", "AOS", "TCA", "LOS", "Duration", "Ground Station"],
)

In [ ]:
accesses_df

## 8. Zone of Interest (France) Access Computation

In [ ]:
# Compute accesses for zone of interest (France with elevation >= 40°)
zone_of_interest_access = access_generator.compute_accesses(
    interval=interval,
    access_targets=[zone_of_interest],
    to_trajectory=satellite_orbit,
)

# Build data for zone of interest
zone_data = []
for access in zone_of_interest_access[0]:  # First (and only) target
    zone_data.append((
        str(access.get_type()),
        repr(access.get_acquisition_of_signal()),
        repr(access.get_time_of_closest_approach()),
        repr(access.get_loss_of_signal()),
        float(access.get_duration().in_seconds()),

    ))

print(f"Zone of interest accesses computed: {len(zone_data)}")

### Zone of Interest Access Results

In [ ]:
# Create DataFrame for zone of interest accesses
zone_of_interest_df = pd.DataFrame(
    data=zone_data,
    columns=["Type", "AOS", "TCA", "LOS", "Duration"],
)

In [ ]:
zone_of_interest_df

## 9. Geometry Computation

In [ ]:
def compute_access_geometry(access, ground_station):
    return [
        compute_time_lla_aer_coordinates(state, ground_station.get_position(), environment)
        for state in satellite_orbit.get_states_at(
            access.get_interval().generate_grid(Duration.seconds(1.0))
        )
    ]

In [ ]:
satellite_orbit_geometry_df = pd.DataFrame(
    [lla.to_vector() for lla in compute_trajectory_geometry(satellite_orbit, interval)],
    columns=["Latitude", "Longitude", "Altitude"],
)

### Satellite Orbit Geometry

In [ ]:
satellite_orbit_geometry_df.head()

### Ground Stations Access Geometry

In [ ]:
# Compute access geometry for all ground stations
access_geometry_dfs = [
    pd.DataFrame(
        compute_access_geometry(access, access_targets[i]),
        columns=[
            "Time",
            "Latitude",
            "Longitude",
            "Altitude",
            "Azimuth",
            "Elevation",
            "Range",
        ],
    )
    for i, target_accesses in enumerate(accesses)
    for access in target_accesses
]

print(f"Ground stations access geometries computed: {len(access_geometry_dfs)}")

### Zone of Interest Access Geometry

In [ ]:
# Compute access geometry for zone of interest (France)
zone_geometry_dfs = [
    pd.DataFrame(
        compute_access_geometry(access, zone_of_interest),
        columns=[
            "Time",
            "Latitude",
            "Longitude",
            "Altitude",
            "Azimuth",
            "Elevation",
            "Range",
        ],
    )
    for access in zone_of_interest_access[0]
]

print(f"Zone of interest access geometries computed: {len(zone_geometry_dfs)}")

In [ ]:
def get_max_elevation(df):
    return df.loc[df["Elevation"].idxmax()]["Elevation"]


## 10. Visualization

In [ ]:
data = []

# Target geometry

for access_target in access_targets:
    data.append(
        dict(
            type="scattergeo",
            lon=[float(access_target.get_lla(earth).get_longitude().in_degrees())],
            lat=[float(access_target.get_lla(earth).get_latitude().in_degrees())],
            mode="markers",
            marker=dict(size=10, color="orange"),
        )
    )

data.append(
    dict(
     type="scattergeo",
        lon=[float(zone_of_interest.get_lla(earth).get_longitude().in_degrees())],
        lat=[float(zone_of_interest.get_lla(earth).get_latitude().in_degrees())],
        mode="markers",
        marker=dict(size=10, color="blue"),
        )
    )

# Orbit geometry

data.append(
    dict(
        type="scattergeo",
        lon=satellite_orbit_geometry_df["Longitude"],
        lat=satellite_orbit_geometry_df["Latitude"],
        mode="lines",
        line=dict(
            width=1,
            color="rgba(0, 0, 0, 0.1)",
        ),
    )
)

# Access geometry

for access_geometry_df in access_geometry_dfs:
    data.append(
        dict(
            type="scattergeo",
            lon=access_geometry_df["Longitude"],
            lat=access_geometry_df["Latitude"],
            mode="lines",
            line=dict(
                width=1,
                color="red",
            ),
        )
    )
for target_geometry_df in zone_geometry_dfs:
    data.append(
        dict(
            type="scattergeo",
            lon=target_geometry_df["Longitude"],
            lat=target_geometry_df["Latitude"],
            mode="lines",
            line=dict(
                width=1,
                color="blue",
            ),
        )
    )



layout = dict(
    title=None,
    showlegend=False,
    width=1200,
    height=600,
    geo=dict(
        showland=True,
        landcolor="rgb(243, 243, 243)",
        countrycolor="rgb(204, 204, 204)",
    ),
)

figure = go.Figure(data=data, layout=layout)

#figure.show("svg")
figure.show()

In [ ]:
# Save zone of interest DataFrame to CSV with readable dates
zone_of_interest_df_export = zone_of_interest_df.copy()
zone_of_interest_df_export['AOS'] = zone_of_interest_df_export['AOS'].apply(lambda x: str(x).split('[')[0].strip())
zone_of_interest_df_export['TCA'] = zone_of_interest_df_export['TCA'].apply(lambda x: str(x).split('[')[0].strip())
zone_of_interest_df_export['LOS'] = zone_of_interest_df_export['LOS'].apply(lambda x: str(x).split('[')[0].strip())
zone_of_interest_df_export.to_csv('zone_of_interest_accesses.csv', index=False)

# Save ground stations accesses DataFrame to CSV with readable dates
accesses_df_export = accesses_df.copy()
accesses_df_export['AOS'] = accesses_df_export['AOS'].apply(lambda x: str(x).split('[')[0].strip())
accesses_df_export['TCA'] = accesses_df_export['TCA'].apply(lambda x: str(x).split('[')[0].strip())
accesses_df_export['LOS'] = accesses_df_export['LOS'].apply(lambda x: str(x).split('[')[0].strip())
accesses_df_export.to_csv('ground_stations_accesses.csv', index=False)

# Create metadata for zone of interest CSV
zone_metadata = """ZONE OF INTEREST ACCESS DATA - METADATA
========================================

File: zone_of_interest_accesses.csv
Generated: Satellite access analysis for France zone of interest
Analysis Period: 2018-01-01 to 2018-01-10 (UTC)

COLUMN DESCRIPTIONS:
-------------------

Type: Access type (Type.Complete indicates full pass with AOS and LOS)

AOS (Acquisition of Signal): Time when satellite is considered in the zone of interest
    - Format: YYYY-MM-DD HH:MM:SS.milliseconds.microseconds.nanoseconds

TCA (Time of Closest Approach): Time when satellite reaches maximum elevation during the pass
    - Format: YYYY-MM-DD HH:MM:SS.milliseconds.microseconds.nanoseconds

LOS (Loss of Signal): Time when satellite is considered leaving the zone of interest
    - Format: YYYY-MM-DD HH:MM:SS.milliseconds.microseconds.nanoseconds

Duration: Total duration of the access window in seconds

VISIBILITY CRITERIA FOR FRANCE (ZONE OF INTEREST):
-------------------------------------------------
- Minimum Elevation: 30° (higher than standard ground stations)
- Azimuth Range: 0° to 360° (all directions)
- Maximum Range: 10,000 km

SATELLITE INFORMATION:
---------------------
- Orbit Type: Sun-synchronous
- Altitude: 500 km
- Epoch: 2018-01-01 12:00:00 UTC

NOTES:
------
- All times are in UTC
- Duration is measured from AOS to LOS
- Total number of access windows in this dataset: """ + str(len(zone_of_interest_df)) + """
"""

with open('zone_of_interest_accesses_metadata.txt', 'w') as f:
        f.write(zone_metadata)

# Create metadata for ground stations CSV
gs_metadata = """GROUND STATIONS ACCESS DATA - METADATA
=======================================

File: ground_stations_accesses.csv
Generated: Satellite access analysis for multiple ground stations
Analysis Period: 2018-01-01 to 2018-01-10 (UTC)

COLUMN DESCRIPTIONS:
-------------------

Type: Access type (Type.Complete indicates full pass with AOS and LOS)

AOS (Acquisition of Signal): Time when satellite first becomes visible above the minimum elevation angle (5°)
    - Format: YYYY-MM-DD HH:MM:SS.milliseconds.microseconds.nanoseconds

TCA (Time of Closest Approach): Time when satellite reaches maximum elevation during the pass
    - Format: YYYY-MM-DD HH:MM:SS.milliseconds.microseconds.nanoseconds

LOS (Loss of Signal): Time when satellite drops below the minimum elevation angle (5°)
    - Format: YYYY-MM-DD HH:MM:SS.milliseconds.microseconds.nanoseconds

Duration: Total duration of the access window in seconds

Ground Station: Name and location of the ground station

GROUND STATIONS:
---------------
""" + '\n'.join([f"- {name}" for name in ground_station_names]) + """

VISIBILITY CRITERIA FOR GROUND STATIONS:
---------------------------------------
- Minimum Elevation: 5° (standard ground station visibility)
- Azimuth Range: 0° to 360° (all directions)
- Maximum Range: 10,000 km

SATELLITE INFORMATION:
---------------------
- Orbit Type: Sun-synchronous
- Altitude: 500 km
- Epoch: 2018-01-01 12:00:00 UTC

NOTES:
------
- All times are in UTC
- Duration is measured from AOS to LOS
- Total number of access windows in this dataset: """ + str(len(accesses_df)) + """
"""

with open('ground_stations_accesses_metadata.txt', 'w') as f:
        f.write(gs_metadata)

print("Tables and metadata saved successfully:")
print("- zone_of_interest_accesses.csv")
print("- zone_of_interest_accesses_metadata.txt")
print("- ground_stations_accesses.csv")
print("- ground_stations_accesses_metadata.txt")